### Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

In [ ]:
pip install pymatgen

In [ ]:
from pymatgen.ext.matproj import MPRester as mpr

### Metals used in alloys

In [ ]:
key = "RBiHILwLoZOoBffRT"

list_of_metals = [ "Al", "Ba", "Zr", "Hg", "Ti", "Ru", "Re", "K", "Rh", "Co", "Mn", "Sr", "Hf", 
 "Y", "Fe", "V", "Ga", "Li", "Na", "Cd", "Ta", "Zn", "Tl", "Mg", "Au", "Sc", "Pt", 
 "Be", "Ir", "Tc", "Ca", "Ni", "Pb", "W", "Cs", "Rb", "Cu", "Nb", "In", "Cr", "Mo", "Pd", "Sn", "Ag" ]

#'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U','Np', 'Pu', 
#'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt',
# 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']

#[ 'Li', 'Be',  'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
 #                 'Ga', 'Ge', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh',
  #                'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Cs', 'Ba',  'Hf', 'Ta', 'W', 'Re', 'Os','Ir', 'Pt', 'Au', ]



print(len(list_of_metals))

44


In [ ]:
mp = mpr(key)

In [ ]:
binary = []  # to store the alloy components

for i in range(len(list_of_metals)):
    for j in range(i+1,len(list_of_metals)):
        binary.append(list_of_metals[i]+"-"+list_of_metals[j])

print(binary)
print(len(binary))

['Al-Ba', 'Al-Zr', 'Al-Hg', 'Al-Ti', 'Al-Ru', 'Al-Re', 'Al-K', 'Al-Rh', 'Al-Co', 'Al-Mn', 'Al-Sr', 'Al-Hf', 'Al-Y', 'Al-Fe', 'Al-V', 'Al-Ga', 'Al-Li', 'Al-Na', 'Al-Cd', 'Al-Ta', 'Al-Zn', 'Al-Tl', 'Al-Mg', 'Al-Au', 'Al-Sc', 'Al-Pt', 'Al-Be', 'Al-Ir', 'Al-Tc', 'Al-Ca', 'Al-Ni', 'Al-Pb', 'Al-W', 'Al-Cs', 'Al-Rb', 'Al-Cu', 'Al-Nb', 'Al-In', 'Al-Cr', 'Al-Mo', 'Al-Pd', 'Al-Sn', 'Al-Ag', 'Ba-Zr', 'Ba-Hg', 'Ba-Ti', 'Ba-Ru', 'Ba-Re', 'Ba-K', 'Ba-Rh', 'Ba-Co', 'Ba-Mn', 'Ba-Sr', 'Ba-Hf', 'Ba-Y', 'Ba-Fe', 'Ba-V', 'Ba-Ga', 'Ba-Li', 'Ba-Na', 'Ba-Cd', 'Ba-Ta', 'Ba-Zn', 'Ba-Tl', 'Ba-Mg', 'Ba-Au', 'Ba-Sc', 'Ba-Pt', 'Ba-Be', 'Ba-Ir', 'Ba-Tc', 'Ba-Ca', 'Ba-Ni', 'Ba-Pb', 'Ba-W', 'Ba-Cs', 'Ba-Rb', 'Ba-Cu', 'Ba-Nb', 'Ba-In', 'Ba-Cr', 'Ba-Mo', 'Ba-Pd', 'Ba-Sn', 'Ba-Ag', 'Zr-Hg', 'Zr-Ti', 'Zr-Ru', 'Zr-Re', 'Zr-K', 'Zr-Rh', 'Zr-Co', 'Zr-Mn', 'Zr-Sr', 'Zr-Hf', 'Zr-Y', 'Zr-Fe', 'Zr-V', 'Zr-Ga', 'Zr-Li', 'Zr-Na', 'Zr-Cd', 'Zr-Ta', 'Zr-Zn', 'Zr-Tl', 'Zr-Mg', 'Zr-Au', 'Zr-Sc', 'Zr-Pt', 'Zr-Be', 'Zr-Ir', 'Zr-Tc', 'Z

### Function to scrape data

In [ ]:
def data_extraction():
    pro = ['energy', 'energy_per_atom', 'volume', 'formation_energy_per_atom', 'unit_cell_formula', 
           'pretty_formula', 'elements', 'nelements', 'e_above_hull', 'band_gap', 'density', 'total_magnetization', 'material_id']

           # above are the properties required
           
    lost = []
    for j in binary:
        data = mp.query(j, pro)
        df = pd.DataFrame(data)
        d1 = mp.query(j, ['spacegroup', 'elasticity', 'material_id']) # these properties are also required
        lst = []
        for i in d1:
            if(True):
                dic = {}
                for k,v in i.items():
                    #print(k,v)
                    if(k == 'material_id'):
                        dic[k] = v  
                    if(k == 'spacegroup'):
                        dic['crystal_system'] = v['crystal_system']
                    if(k == 'elasticity' and v != None):
                        for h,m in v.items():
                            if(h not in ['compliance_tensor', 'elastic_tensor', 'warnings', 'elastic_tensor_original', 'nsites']):
                                dic[h] = m
                            if(h == 'elastic_tensor' ):
                                dic['C11'] = m[0][0]
                                dic['C22'] = m[1][1]
                                dic['C33'] = m[2][2]
                                dic['C44'] = m[3][3]
                                dic['C55'] = m[4][4]
                                dic['C66'] = m[5][5]
                                dic['C12'] = m[0][1]
                                dic['C13'] = m[0][2]
                                dic['C23'] = m[1][2]                           
            lst.append(dic)
        df2 = pd.DataFrame(lst)
        try:
            df_t = pd.merge(df, df2, on = "material_id")
            lost.append(df_t)
        except:
            pass
    result = pd.concat(lost)
    return result



In [ ]:
data = data_extraction()

In [ ]:
print(len(data))

4640


In [ ]:
data.head(10)

,energy,energy_per_atom,volume,formation_energy_per_atom,unit_cell_formula,pretty_formula,elements,nelements,e_above_hull,band_gap,...,C33,C44,C55,C66,C12,C13,C23,homogeneous_poisson,poisson_ratio,universal_anisotropy
0,-56.798224,-3.155457,582.448536,-0.221706,"{'Ba': 8.0, 'Al': 10.0}",Ba4Al5,"[Al, Ba]",2,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-52.652910,-3.290807,472.594457,-0.230208,"{'Ba': 6.0, 'Al': 10.0}",Ba3Al5,"[Al, Ba]",2,0.007396,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-54.811339,-3.224196,522.734581,-0.230752,"{'Ba': 7.0, 'Al': 10.0}",Ba7Al10,"[Al, Ba]",2,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-19.808774,-1.980877,619.211457,0.303414,"{'Ba': 8.0, 'Al': 2.0}",Ba4Al,"[Al, Ba]",2,0.383228,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-66.473703,-3.323685,549.874873,-0.217421,"{'Ba': 7.0, 'Al': 13.0}",Ba7Al13,"[Al, Ba]",2,0.024842,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-23.633336,-2.954167,297.612603,-0.121894,"{'Ba': 4.0, 'Al': 4.0}",BaAl,"[Al, Ba]",2,0.077641,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-18.107169,-3.621434,118.890829,-0.241179,"{'Ba': 1.0, 'Al': 4.0}",BaAl4,"[Ba, Al]",2,0.000000,0.0,...,108.0,36.0,36.0,37.0,27.0,31.0,31.0,0.22,0.22,0.06
7,-20.013340,-3.335557,155.720125,-0.198849,"{'Ba': 2.0, 'Al': 4.0}",BaAl2,"[Ba, Al]",2,0.044320,0.0,...,85.0,32.0,32.0,32.0,21.0,21.0,21.0,0.20,0.20,0.00
8,-204.964709,-3.360077,1694.276245,-0.243333,"{'Ba': 21.0, 'Al': 40.0}",Ba21Al40,"[Al, Ba]",2,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,-43.588191,-7.264698,126.776938,-0.317706,"{'Zr': 4.0, 'Al': 2.0}",Zr2Al,"[Zr, Al]",2,0.054448,0.0,...,132.0,53.0,61.0,61.0,85.0,85.0,81.0,0.32,0.32,0.97


In [ ]:
data.keys()

Index(['energy', 'energy_per_atom', 'volume', 'formation_energy_per_atom',
       'unit_cell_formula', 'pretty_formula', 'elements', 'nelements',
       'e_above_hull', 'band_gap', 'density', 'total_magnetization',
       'material_id', 'crystal_system', 'G_Reuss', 'G_VRH', 'G_Voigt',
       'G_Voigt_Reuss_Hill', 'K_Reuss', 'K_VRH', 'K_Voigt',
       'K_Voigt_Reuss_Hill', 'elastic_anisotropy', 'C11', 'C22', 'C33', 'C44',
       'C55', 'C66', 'C12', 'C13', 'C23', 'homogeneous_poisson',
       'poisson_ratio', 'universal_anisotropy'],
      dtype='object')

In [ ]:
data.dropna(how = 'any', inplace=True) # to drop all rows having even one NA values

In [ ]:
print(len(data))

1488


In [ ]:
data.head()

,energy,energy_per_atom,volume,formation_energy_per_atom,unit_cell_formula,pretty_formula,elements,nelements,e_above_hull,band_gap,...,C33,C44,C55,C66,C12,C13,C23,homogeneous_poisson,poisson_ratio,universal_anisotropy
6,-18.107169,-3.621434,118.890829,-0.241179,"{'Ba': 1.0, 'Al': 4.0}",BaAl4,"[Ba, Al]",2,0.000000,0.0,...,108.0,36.0,36.0,37.0,27.0,31.0,31.0,0.22,0.22,0.06
7,-20.013340,-3.335557,155.720125,-0.198849,"{'Ba': 2.0, 'Al': 4.0}",BaAl2,"[Ba, Al]",2,0.044320,0.0,...,85.0,32.0,32.0,32.0,21.0,21.0,21.0,0.20,0.20,0.00
0,-43.588191,-7.264698,126.776938,-0.317706,"{'Zr': 4.0, 'Al': 2.0}",Zr2Al,"[Zr, Al]",2,0.054448,0.0,...,132.0,53.0,61.0,61.0,85.0,85.0,81.0,0.32,0.32,0.97
1,-43.464326,-5.433041,139.934873,-0.486934,"{'Zr': 2.0, 'Al': 6.0}",ZrAl3,"[Zr, Al]",2,0.000000,0.0,...,201.0,81.0,81.0,99.0,64.0,45.0,45.0,0.18,0.18,0.06
4,-61.878187,-6.187819,188.086769,-0.521393,"{'Zr': 4.0, 'Al': 6.0}",Zr2Al3,"[Zr, Al]",2,0.000000,0.0,...,202.0,71.0,77.0,51.0,52.0,48.0,72.0,0.24,0.24,0.16


In [ ]:
data.to_excel("ME793_data.xlsx", index=False)
data.to_csv("ME793_data.csv", index=False)

In [ ]:
three_com = []  # to store components of three component alloys

for i in range(len(list_of_metals)):
    for j in range(i+1,len(list_of_metals)):
        for k in range(j+1, len(list_of_metals)):
            three_com.append(list_of_metals[i]+"-"+list_of_metals[j]+"-"+list_of_metals[k])

print(three_com)
print(len(three_com))

['Al-Ba-Zr', 'Al-Ba-Hg', 'Al-Ba-Ti', 'Al-Ba-Ru', 'Al-Ba-Re', 'Al-Ba-K', 'Al-Ba-Rh', 'Al-Ba-Co', 'Al-Ba-Mn', 'Al-Ba-Sr', 'Al-Ba-Hf', 'Al-Ba-Y', 'Al-Ba-Fe', 'Al-Ba-V', 'Al-Ba-Ga', 'Al-Ba-Li', 'Al-Ba-Na', 'Al-Ba-Cd', 'Al-Ba-Ta', 'Al-Ba-Zn', 'Al-Ba-Tl', 'Al-Ba-Mg', 'Al-Ba-Au', 'Al-Ba-Sc', 'Al-Ba-Pt', 'Al-Ba-Be', 'Al-Ba-Ir', 'Al-Ba-Tc', 'Al-Ba-Ca', 'Al-Ba-Ni', 'Al-Ba-Pb', 'Al-Ba-W', 'Al-Ba-Cs', 'Al-Ba-Rb', 'Al-Ba-Cu', 'Al-Ba-Nb', 'Al-Ba-In', 'Al-Ba-Cr', 'Al-Ba-Mo', 'Al-Ba-Pd', 'Al-Ba-Sn', 'Al-Ba-Ag', 'Al-Zr-Hg', 'Al-Zr-Ti', 'Al-Zr-Ru', 'Al-Zr-Re', 'Al-Zr-K', 'Al-Zr-Rh', 'Al-Zr-Co', 'Al-Zr-Mn', 'Al-Zr-Sr', 'Al-Zr-Hf', 'Al-Zr-Y', 'Al-Zr-Fe', 'Al-Zr-V', 'Al-Zr-Ga', 'Al-Zr-Li', 'Al-Zr-Na', 'Al-Zr-Cd', 'Al-Zr-Ta', 'Al-Zr-Zn', 'Al-Zr-Tl', 'Al-Zr-Mg', 'Al-Zr-Au', 'Al-Zr-Sc', 'Al-Zr-Pt', 'Al-Zr-Be', 'Al-Zr-Ir', 'Al-Zr-Tc', 'Al-Zr-Ca', 'Al-Zr-Ni', 'Al-Zr-Pb', 'Al-Zr-W', 'Al-Zr-Cs', 'Al-Zr-Rb', 'Al-Zr-Cu', 'Al-Zr-Nb', 'Al-Zr-In', 'Al-Zr-Cr', 'Al-Zr-Mo', 'Al-Zr-Pd', 'Al-Zr-Sn', 'Al-Zr-Ag', 'Al-Hg-Ti',

### Function to extract 3 component alloy data

In [ ]:
def data_extraction_three_component():
    pro = ['energy', 'energy_per_atom', 'volume', 'formation_energy_per_atom', 'unit_cell_formula', 
           'pretty_formula', 'elements', 'nelements', 'e_above_hull', 'band_gap', 'density', 'total_magnetization', 'material_id']
           
    lost = []
    f=0
    for j in three_com:
        try:
            data1 = mp.query(j, pro)
        except:
            pass
        if(len(data1) != 0):
            df = pd.DataFrame(data1)
            try:
                d2 = mp.query(j, ['spacegroup', 'elasticity', 'material_id'])
            except:
                pass
            if(len(d2) != 0):
                lst = []
                for i in d2:
                    if(True):
                        dic = {}
                        for k,v in i.items():
                            #print(k,v)
                            if(k == 'material_id'):
                                dic[k] = v  
                            if(k == 'spacegroup'):
                                dic['crystal_system'] = v['crystal_system']
                            if(k == 'elasticity' and v != None):
                                for h,m in v.items():
                                    if(h not in ['compliance_tensor', 'elastic_tensor', 'warnings', 'elastic_tensor_original', 'nsites']):
                                        dic[h] = m
                                    if(h == 'elastic_tensor' ):
                                        dic['C11'] = m[0][0]
                                        dic['C22'] = m[1][1]
                                        dic['C33'] = m[2][2]
                                        dic['C44'] = m[3][3]
                                        dic['C55'] = m[4][4]
                                        dic['C66'] = m[5][5]
                                        dic['C12'] = m[0][1]
                                        dic['C13'] = m[0][2]
                                        dic['C23'] = m[1][2]                           
                    lst.append(dic)
                df2 = pd.DataFrame(lst)
                try:
                    df_t = pd.merge(df, df2, on = "material_id")
                    lost.append(df_t)
                    
                except:
                    pass
        print(f)
        print(j)
        f = f+1
    result = pd.concat(lost)
    return result

In [ ]:
test_data = data_extraction_three_component()

In [ ]:
test_data.dropna(how='any', inplace= True)  # droping NA value rows

In [ ]:
test_data.head()

,energy,energy_per_atom,volume,formation_energy_per_atom,unit_cell_formula,pretty_formula,elements,nelements,e_above_hull,band_gap,...,C33,C44,C55,C66,C12,C13,C23,homogeneous_poisson,poisson_ratio,universal_anisotropy
0,-56.369357,-4.697446,219.297872,-0.316801,"{'Ba': 1.0, 'Al': 9.0, 'Fe': 2.0}",BaAl9Fe2,"[Ba, Al, Fe]",3,0.000000,0.0,...,126.0,69.0,69.0,31.0,65.0,51.0,51.0,0.26,0.26,0.70
0,-51.522250,-4.293521,217.985996,-0.361121,"{'Ba': 1.0, 'Al': 9.0, 'Ni': 2.0}",BaAl9Ni2,"[Ba, Al, Ni]",3,0.004534,0.0,...,154.0,52.0,52.0,36.0,64.0,38.0,38.0,0.26,0.26,0.23
0,-35.674179,-3.243107,363.812653,-0.330805,"{'Ba': 5.0, 'Al': 5.0, 'Pb': 1.0}",Ba5Al5Pb,"[Ba, Al, Pb]",3,0.000000,0.0,...,55.0,16.0,16.0,20.0,11.0,20.0,20.0,0.25,0.25,0.09
1,-24.839717,-3.548531,254.771167,-0.515295,"{'Ba': 3.0, 'Al': 2.0, 'Sn': 2.0}",Ba3(AlSn)2,"[Ba, Al, Sn]",3,0.006500,0.0,...,72.0,15.0,27.0,16.0,19.0,15.0,16.0,0.24,0.24,0.33
2,-36.148251,-3.286205,360.950849,-0.348475,"{'Ba': 5.0, 'Al': 5.0, 'Sn': 1.0}",Ba5Al5Sn,"[Ba, Al, Sn]",3,0.000000,0.0,...,56.0,17.0,17.0,21.0,12.0,20.0,20.0,0.25,0.25,0.08


In [ ]:
test_data.keys()

Index(['energy', 'energy_per_atom', 'volume', 'formation_energy_per_atom',
       'unit_cell_formula', 'pretty_formula', 'elements', 'nelements',
       'e_above_hull', 'band_gap', 'density', 'total_magnetization',
       'material_id', 'crystal_system', 'G_Reuss', 'G_VRH', 'G_Voigt',
       'G_Voigt_Reuss_Hill', 'K_Reuss', 'K_VRH', 'K_Voigt',
       'K_Voigt_Reuss_Hill', 'elastic_anisotropy', 'C11', 'C22', 'C33', 'C44',
       'C55', 'C66', 'C12', 'C13', 'C23', 'homogeneous_poisson',
       'poisson_ratio', 'universal_anisotropy'],
      dtype='object')

In [ ]:
len(test_data)

1260

In [ ]:
test_data.to_excel("ME793_three_component_data.xlsx", index=False)
test_data.to_csv("ME793_three_component_data.csv", index=False)